In [27]:
import sys
import os
from os.path import exists
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from energy_functions import calcE, calcDPFE, calcGHBind
sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/Utils')
sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/QMkit')
from dft_utils import checkvibs, check, no_to_symbol
from gen_3D import get_xyz_from_xyz_arr,combine_labels_and_xyz

import cclib
import plot as pl
import pathlib
from tqdm import tqdm
from datetime import datetime, timedelta
from rdkit import Chem

In [2]:
#Define Current directory
cwd= '/home/gridsan/jmaalouf/vHTP/Code/ylide/Library'

# Load Data

In [3]:
df=pd.read_csv('../sheets/Ylides_YlideRads_Cleaned.csv')
y=df['Ylides'].to_list()
yr=df['Ylides Rad'].to_list()
yh=df['Ylides H'].to_list()

## Get Num Atoms

In [4]:
num_atoms=[]
num_atoms_total=[]
for i, s in enumerate(tqdm(y)):
    mol=Chem.MolFromSmiles(s)
    num_atoms.append(mol.GetNumAtoms())
    mol=Chem.AddHs(mol)
    num_atoms_total.append(mol.GetNumAtoms())

100%|██████████| 19097/19097 [00:01<00:00, 10149.81it/s]


In [8]:
with open(f"../pickled_data/num_atoms.pkl", "wb") as file: 
    pickle.dump(num_atoms, file)
    
with open(f"../pickled_data/num_atoms_total.pkl", "wb") as file: 
    pickle.dump(num_atoms_total, file)

# Check for Failed Calcs

In [17]:
trueChoices = ['true', 't','T']
falseChoices = ['false', 'f','F']

inp=input("Do you want to resubmit the memory allocation errors? (T/F)").lower()

assert inp in trueChoices or inp in falseChoices
if inp in trueChoices:
    resubmit=True
elif inp in falseChoices:
    resubmit=False

orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'

failed_dict={}
geom_failed_dict={}

functional='M062X'
basis= 'Def2SVPD'
solvorgas='solv'
solvmethod='SMD'
solvent='acetonitrile'

if solvorgas== 'gas':
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
elif solvorgas =='solv':
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt']]
    #ylide_type=[['ylide','_gas_preopt'],['ylide_h','_gas_preopt']]

start=0
stop=10000
n=stop-start
error_occurred=Falsez

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    failed_dict[f'{y}{sub_suff}']=[]
    geom_failed_dict[f'{y}{sub_suff}']=[]
    
    for i in tqdm(range(start,stop)):
    #for i in tqdm(failed_dict_08_23[f'{y}{sub_suff}']):
        n_str=str(i).zfill(7)
        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
        
        par_path=path.parent.absolute()
        
        if exists(path):
            
            if check(path, 'Normal termination of Gaussian', count=2)==False:
                error_occurred=True
                
                if check(str(path),'galloc:  could not allocate memory.'):
                    
                    #print(f'MEMRORY ALLOCATION ERROR WITH {y}{sub_suff} MOLECULE {i}')
                    failed_dict[f'{y}{sub_suff}'].append(i)
                    
                    if resubmit:
                        os.chdir(par_path)
                        subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                        os.chdir(orig_dir)
                                
                elif check(str(path),'Converged',120):
                    #print(f'GEOMETRY OPT ERROR WITH {y}{sub_suff} MOLECULE {i}')
                    #failed_dict[f'{y}{sub_suff}'].append(i)
                    geom_failed_dict[f'{y}{sub_suff}'].append(i)
                    
                else:
                    #Try one more time to extract the data because there could still be some optimized geometry
                    #steps that can be extracted
                    #print(f'UNKNOWN ERROR, {y}{sub_suff} MOLECULE {i} CALCULATION DID NOT FINSIH')
                    failed_dict[f'{y}{sub_suff}'].append(i)
                    
                    if resubmit:
                        os.chdir(par_path)
                        #subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                        os.chdir(orig_dir)
        else:
            #print(f'{y} MOLECULE {i}{sub_suff} HAS NOT BEEN SUBMITTED')
            failed_dict[f'{y}{sub_suff}'].append(i)
            
            if resubmit:
                os.chdir(par_path)
                subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                os.chdir(orig_dir)
                
    if error_occurred==False:
        print(f'NO ERRORS WITH {y}{sub_suff}')
        
        
    error_occurred=False
    print('')
    


Do you want to resubmit the memory allocation errors? (T/F) f


  9%|▊         | 7/82 [00:00<00:01, 65.96it/s]

  8%|▊         | 2/25 [00:00<00:01, 18.77it/s]

100%|██████████| 25/25 [00:01<00:00, 14.68it/s]

### Save Dictionaries

In [14]:
with open(f"/data1/groups/manthiram_lab/vHTP/Code/ylide/Library/python_scripts/dicts/failed_dict_{solvorgas}_{'0823'}.pkl", "wb") as file: 
    pkl.dump(failed_dict, file)

In [15]:
with open(f"/data1/groups/manthiram_lab/vHTP/Code/ylide/Library/python_scripts/dicts/failed_dict_{solvorgas}_{'0823'}.pkl","rb") as file:
    pickle_off = pkl.Unpickler(file)
    failed_dict_08_23= pickle_off.load()

In [ ]:
with open(f"../pickled_data/failed_dict_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [22]:
with open(f"../pickled_data/geom_failed_dict_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(geom_failed_dict, file)

#### Visualize contents of dictionaries

In [18]:
for key, value in failed_dict.items():
    print(f'{key} has {len(value)} failures and a fail percentage of {len(value)/n*100}%')

ylide_gas_preopt has 13 failures and a fail percentage of 0.13%
ylide_rad_gas_preopt has 82 failures and a fail percentage of 0.8200000000000001%
ylide_h_gas_preopt has 25 failures and a fail percentage of 0.25%


In [19]:
for key, value in geom_failed_dict.items():
    print(f'{key} has {len(value)} failures and a fail percentage of {len(value)/n*100}%')

ylide_gas_preopt has 0 failures and a fail percentage of 0.0%
ylide_rad_gas_preopt has 0 failures and a fail percentage of 0.0%
ylide_h_gas_preopt has 0 failures and a fail percentage of 0.0%


#### Wall Times For Different Triples Conditons

[5,1,1] wall_time: [1816, 2098, 2524, 2864, 1405, 1256, 1628, 1559, 1741, 1783] <br>
[10,1,2] wall_time: [[1800, 0, 0, 0, 1398, 1248, 1618, 1560, 1732, 1787]] <br>
[5,2,1] wall_time: [3784, 0, 5337, 0, 0, 2829, 0, 3480, 3763, 0] <br>

There seems to be no effect from number of threads, it seems that we still have an effect from RAM and it scales pretty linearly so I will use 1 node per calculation

In [20]:
print('CALCULATING REDOX POTENTIALS')    
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,7]=calcE(data[i,3],data[i,1])
    
    
print('CALCULATING DPFE')
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,8]=calcDPFE(data[i,5],data[i,1],solvent=solvent)

print('CALCULATING GHBind ')
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,9]=calcGHBind(data[i,5],data[i,3],solvent='gas')
    

100%|██████████| 1000/1000 [00:00<00:00, 703270.29it/s]

CALCULATING REDOX POTENTIALS
CALCULATING DPFE
CALCULATING GHBind 
